<a href="https://colab.research.google.com/github/Pinakbakshi/Algo-trading/blob/main/BTC_data_Analysis_using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow
!pip install keras


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

In [ ]:
# Load the dataset
file_path = '/content/BTC-USDT_30m.csv'
data = pd.read_csv(file_path)

In [ ]:
# Preprocessing the data
data['timestamp'] = pd.to_datetime(data['timestamp'])
data.set_index('timestamp', inplace=True)

In [ ]:
# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

In [ ]:
# Create sequences for LSTM
def create_sequences(data, seq_length):
    X = []
    y = []
    for i in range(len(data) - seq_length - 1):
        X.append(data[i:(i + seq_length), :-1])
        y.append(data[i + seq_length, 3])  # Close price as the target
    return np.array(X), np.array(y)

seq_length = 60  # Using 60 time steps
X, y = create_sequences(scaled_data, seq_length)


In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Build the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=1))

model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
# Train the model
history = model.fit(X_train, y_train, epochs=2, batch_size=32, validation_split=0.1, shuffle=False)

Epoch 1/2
2694/2694 [==============================] - 192s 69ms/step - loss: 0.0011 - val_loss: 2.7573e-05
Epoch 2/2
2694/2694 [==============================] - 148s 55ms/step - loss: 5.1706e-04 - val_loss: 2.7343e-05


In [ ]:
# Predict on the test set
y_pred = model.predict(X_test)

749/749 [==============================] - 13s 17ms/step


In [ ]:
y_pred = model.predict(X_test)

# Reshape y_test and y_pred for inverse transform
y_test_reshaped = y_test.reshape(-1, 1)
y_pred_reshaped = y_pred.reshape(-1, 1)

# Inverse transform to get actual values
y_test_inv = scaler.inverse_transform(np.concatenate((np.zeros((y_test_reshaped.shape[0], X_test.shape[2])), y_test_reshaped), axis=1))[:, -1]
y_pred_inv = scaler.inverse_transform(np.concatenate((np.zeros((y_pred_reshaped.shape[0], X_test.shape[2])), y_pred_reshaped), axis=1))[:, -1]

# Evaluate the model
precision = precision_score(y_test_inv > 0, y_pred_inv > 0, zero_division=1)
recall = recall_score(y_test_inv > 0, y_pred_inv > 0, zero_division=1)
f1 = f1_score(y_test_inv > 0, y_pred_inv > 0, zero_division=1)

print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

749/749 [==============================] - 16s 21ms/step
Precision: 1.0
Recall: 1.0
F1 Score: 1.0


In [ ]:
# Evaluate the model
precision = precision_score(y_test_inv > 0, y_pred_inv > 0)
recall = recall_score(y_test_inv > 0, y_pred_inv > 0)
f1 = f1_score(y_test_inv > 0, y_pred_inv > 0)


print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

Precision: 1.0
Recall: 1.0
F1 Score: 1.0


In [ ]:
# Generate buy/sell signals
signals = []
for i in range(1, len(y_pred_inv)):
    if y_pred_inv[i] > y_test_inv[i-1]:
        signals.append("BUY")
    else:
        signals.append("SELL")

# Add signals to the original dataset
data['Signal'] = np.nan
data['Signal'].iloc[-len(signals):] = signals

# Display the latest signals
print(data[['open', 'high', 'low', 'close', 'Signal']].tail())

                         open      high       low     close Signal
timestamp                                                         
2024-06-22 05:30:00  64361.49  64427.98  64334.01  64424.21   SELL
2024-06-22 06:00:00  64424.21  64466.00  64362.06  64408.56   SELL
2024-06-22 06:30:00  64408.57  64530.61  64370.00  64430.00    BUY
2024-06-22 07:00:00  64430.01  64444.00  64324.67  64408.19    BUY
2024-06-22 07:30:00  64408.18  64490.79  64402.54  64465.29   SELL


In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

# Load the dataset
file_path = '/content/BTC-USDT_30m.csv'
data = pd.read_csv(file_path)

# Preprocessing the data
data['timestamp'] = pd.to_datetime(data['timestamp'])
data.set_index('timestamp', inplace=True)

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

# Create sequences for LSTM
def create_sequences(data, seq_length):
    X = []
    y = []
    for i in range(len(data) - seq_length - 1):
        X.append(data[i:(i + seq_length), :-1])
        y.append(data[i + seq_length, 3])  # Close price as the target
    return np.array(X), np.array(y)

seq_length = 60  # Using 60 time steps
X, y = create_sequences(scaled_data, seq_length)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=1))

model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1, shuffle=False)

y_pred = model.predict(X_test)

# Reshape y_test and y_pred for inverse transform
y_test_reshaped = y_test.reshape(-1, 1)
y_pred_reshaped = y_pred.reshape(-1, 1)

# Inverse transform to get actual values
y_test_inv = scaler.inverse_transform(np.concatenate((np.zeros((y_test_reshaped.shape[0], X_test.shape[2])), y_test_reshaped), axis=1))[:, -1]
y_pred_inv = scaler.inverse_transform(np.concatenate((np.zeros((y_pred_reshaped.shape[0], X_test.shape[2])), y_pred_reshaped), axis=1))[:, -1]

# Evaluate the model
precision = precision_score(y_test_inv > 0, y_pred_inv > 0, zero_division=1)
recall = recall_score(y_test_inv > 0, y_pred_inv > 0, zero_division=1)
f1 = f1_score(y_test_inv > 0, y_pred_inv > 0, zero_division=1)

print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')


# Generate buy/sell signals
signals = []
for i in range(1, len(y_pred_inv)):
    if y_pred_inv[i] > y_test_inv[i-1]:
        signals.append("BUY")
    else:
        signals.append("SELL")

# Add signals to the original dataset
data['Signal'] = np.nan
data['Signal'].iloc[-len(signals):] = signals

# Display the latest signals
print(data[['open', 'high', 'low', 'close', 'Signal']].tail())


Epoch 1/10
2694/2694 [==============================] - 174s 62ms/step - loss: 9.4004e-04 - val_loss: 2.2098e-04
Epoch 2/10
2694/2694 [==============================] - 165s 61ms/step - loss: 5.1195e-04 - val_loss: 1.2337e-04
Epoch 3/10
2694/2694 [==============================] - 170s 63ms/step - loss: 4.7958e-04 - val_loss: 2.7750e-05
Epoch 4/10
2694/2694 [==============================] - 174s 65ms/step - loss: 4.7100e-04 - val_loss: 2.5483e-05
Epoch 5/10
2694/2694 [==============================] - 169s 63ms/step - loss: 4.6877e-04 - val_loss: 4.2417e-05
Epoch 6/10
2694/2694 [==============================] - 170s 63ms/step - loss: 4.6635e-04 - val_loss: 4.2129e-05
Epoch 7/10
2694/2694 [==============================] - 172s 64ms/step - loss: 4.5361e-04 - val_loss: 3.7636e-05
Epoch 8/10
2694/2694 [==============================] - 170s 63ms/step - loss: 4.4591e-04 - val_loss: 1.3583e-04
Epoch 9/10
2694/2694 [==============================] - 172s 64ms/step - loss: 4.4555e-04 - val_